In [ ]:
from IPython.display import Audio
import soundfile as sf
from scipy.signal import spectrogram
import matplotlib.pyplot as plt
import os
import librosa
import librosa.display
from tqdm import tqdm_notebook
from pathlib import Path
import shutil
import numpy as np
import scipy
from random import shuffle
import matplotlib
import csv

In [ ]:
header = 'filename rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 13):
    header += f' chroma_stft{i}'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [ ]:
for split in ['train','dev','eval']:
    with open(f'dataset_{split}.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
    labels = 'real fake'.split()
    for label in labels:
        for filename in tqdm_notebook(os.listdir(f'DATASET PATH/{split}/{label}')):
            songname = f'audio_biggerthan2sec/{split}/{label}/{filename}'
            y, sr = librosa.load(songname)
            rmse = librosa.feature.rms(y=y)
            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            zcr = librosa.feature.zero_crossing_rate(y)
            mfcc = librosa.feature.mfcc(y=y, sr=sr)
            to_append = f'{filename} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'
            for c in chroma_stft:
                to_append += f' {np.mean(c)}'
            for e in mfcc:
                to_append += f' {np.mean(e)}'
            to_append += f' {label}'
            with open(f'dataset_{split}.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(to_append.split())